In [1]:
import os

In [2]:
%pwd

'c:\\Users\\parva\\OneDrive\\Desktop\\Assignments\\Chicken_Disease\\Chicken_Disease_Classification\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\parva\\OneDrive\\Desktop\\Assignments\\Chicken_Disease\\Chicken_Disease_Classification'

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir:Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [6]:
from Chicken_Disease_Classification.constants import *
from Chicken_Disease_Classification.utils.common import read_yaml, create_directories

In [12]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config

In [9]:
import os 
import urllib.request as request
import zipfile
from Chicken_Disease_Classification import logger
from Chicken_Disease_Classification.utils.common import get_size

In [10]:
class DataIngestion:
    def __init__(self,config:DataIngestionConfig):
        self.config = config

    def donwload_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} donload! with folowing info : \n {headers}")
        else:
            logger.info(f"file already exists of size: {get_size(Path(self.config.local_data_file))}")
        
    def extract_zip_file(self):
        """
        zip_file_path:str
        Extracts the zip file into the data directory
        Functin returns None

          """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [13]:
#creating pipeline

try:
    config= ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config = data_ingestion_config)
    data_ingestion.donwload_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2023-10-05 12:13:42,004: INFO: common: yaml file: config\config.yaml loaded successfully]
[2023-10-05 12:13:42,007: INFO: common: yaml file: params.yaml loaded successfully]
[2023-10-05 12:13:42,008: INFO: common: created directory at: artifacts]
[2023-10-05 12:13:42,009: INFO: common: created directory at: artifacts/data_ingestion]
[2023-10-05 12:13:46,921: INFO: 3049578050: artifacts/data_ingestion/data.zip donload! with folowing info : 
 Connection: close
Content-Length: 11616915
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "adf745abc03891fe493c3be264ec012691fe3fa21d861f35a27edbe6d86a76b1"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 2454:14A2:368F2D:43A36D:651E5B18
Accept-Ranges: bytes
Date: Thu, 05 Oct 2023 06:43:45 GMT
Via: 1.1 varnish
X-Served-By: cache-maa10224-MAA
X-Cache: MI